In [1]:
pip install cmake dlib opencv-python numpy face-recognition matplotlib pillow


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import os

def detect_and_crop_faces(input_folder, output_folder):
    # Load OpenCV's Haar Cascade for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate over all subfolders in the input folder
    for person_folder in os.listdir(input_folder):
        person_input_path = os.path.join(input_folder, person_folder)

        # Check if it's a directory (skip files)
        if os.path.isdir(person_input_path):
            person_output_path = os.path.join(output_folder, person_folder)

            if not os.path.exists(person_output_path):
                os.makedirs(person_output_path)

            # Process all images in each person's folder
            for filename in os.listdir(person_input_path):
                image_path = os.path.join(person_input_path, filename)
                img = cv2.imread(image_path)

                print(f"Image being Cropped: {image_path}")
                
                # Check if image is read successfully
                if img is not None:
                    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    
                    # Detect faces in the image
                    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
                    
                    # Crop and save each detected face
                    for (x, y, w, h) in faces:
                        face_img = img[y:y+h, x:x+w]
                        face_filename = os.path.join(person_output_path, filename)
                        cv2.imwrite(face_filename, face_img)

# Example usage for detecting and cropping faces from multiple folders
input_folder = '/Users/narendraraj/fr/dataset'
output_folder = '/Users/narendraraj/fr/cropped_dataset'

detect_and_crop_faces(input_folder, output_folder)

Image being Cropped: /Users/narendraraj/fr/dataset/Karan/IMG-20241018-WA0022.jpg
Image being Cropped: /Users/narendraraj/fr/dataset/Karan/IMG-20241018-WA0023.jpg
Image being Cropped: /Users/narendraraj/fr/dataset/Karan/karan.jpeg.jpg
Image being Cropped: /Users/narendraraj/fr/dataset/Karan/IMG-20241018-WA0021.jpg
Image being Cropped: /Users/narendraraj/fr/dataset/Karan/IMG-20241018-WA0024.jpg
Image being Cropped: /Users/narendraraj/fr/dataset/Karan/WhatsApp Image 2024-10-18 at 11.20.57_f8a09022.jpg
Image being Cropped: /Users/narendraraj/fr/dataset/Karan/IMG-20241018-WA0019.jpg
Image being Cropped: /Users/narendraraj/fr/dataset/Manish/.DS_Store
Image being Cropped: /Users/narendraraj/fr/dataset/Manish/manish.jpeg.jpg
Image being Cropped: /Users/narendraraj/fr/dataset/Manish/IMG-20241018-WA0016.jpg
Image being Cropped: /Users/narendraraj/fr/dataset/Manish/IMG-20241018-WA0014.jpg
Image being Cropped: /Users/narendraraj/fr/dataset/Manish/IMG-20241018-WA0015.jpg
Image being Cropped: /Users

In [4]:
# The lines `import cv2`, `import numpy as np`, and `import face_recognition` are importing necessary
# libraries in Python for image processing and face recognition tasks.
import cv2
import numpy as np
import face_recognition
import os
import pickle
from datetime import datetime

In [6]:
# Path to the directory with images for encoding
path = "cropped_dataset"
images = []
classNames = []
# List all items in the specified path and store images and class names
def makeClasses():
    for cls in os.listdir(path):
        cls_path = os.path.join(path, cls)
        if os.path.isdir(cls_path):
            for img in os.listdir(cls_path):
                if not img.endswith('.DS_Store'):
                    images.append(os.path.join(cls_path, img))
                    classNames.append(cls)
    return classNames

classNames = makeClasses()

print("Image files:", images)
print("Class names:", classNames)

Image files: ['cropped_dataset/Karan/IMG-20241018-WA0022.jpg', 'cropped_dataset/Karan/IMG-20241018-WA0023.jpg', 'cropped_dataset/Karan/karan.jpeg.jpg', 'cropped_dataset/Karan/IMG-20241018-WA0021.jpg', 'cropped_dataset/Karan/WhatsApp Image 2024-10-18 at 11.20.57_f8a09022.jpg', 'cropped_dataset/Karan/IMG-20241018-WA0019.jpg', 'cropped_dataset/Manish/manish.jpeg.jpg', 'cropped_dataset/Manish/IMG-20241018-WA0016.jpg', 'cropped_dataset/Manish/IMG-20241018-WA0014.jpg', 'cropped_dataset/Manish/IMG-20241018-WA0012.jpg', 'cropped_dataset/Sathwik/IMG-20241028-WA0025.jpg', 'cropped_dataset/Sathwik/IMG-20241028-WA0018.jpg', 'cropped_dataset/Sathwik/IMG-20241028-WA0024.jpg', 'cropped_dataset/Sathwik/IMG-20241028-WA0020.jpg', 'cropped_dataset/Sathwik/IMG-20241028-WA0021.jpg', 'cropped_dataset/Sathwik/IMG-20241028-WA0023.jpg', 'cropped_dataset/Sathwik/IMG-20241028-WA0022.jpg', 'cropped_dataset/Sathwik/IMG-20241028-WA0013.jpg', 'cropped_dataset/Sathwik/IMG-20241028-WA0012.jpg', 'cropped_dataset/Sathwi

In [7]:
# Function for encoding faces
print("encoding start")
counter = 0
def findEncodings(images):
    global counter
    encodeList = []
    for img_path in images:
        img = cv2.imread(img_path)
        if img is None:
            print(f"Error loading image: {img_path}")
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encodes = face_recognition.face_encodings(img)
        if encodes:
            encodeList.append(encodes[0]) # Use the first encoding if present
            counter+=1      
            print(f'encoded image: {counter}')
        else:
            print(f"No face found in image: {img_path}")
    
    # Validation message after processing all images
    if encodeList:
        print(f"Encoding complete. {len(encodeList)} face(s) encoded successfully.")
    else:
        print("Encoding complete. No faces were found in the provided images.")
    
    return encodeList

encodeListKnown = findEncodings(images)

encoding start
encoded image: 1
encoded image: 2
encoded image: 3
encoded image: 4
encoded image: 5
encoded image: 6
encoded image: 7
No face found in image: cropped_dataset/Manish/IMG-20241018-WA0016.jpg
No face found in image: cropped_dataset/Manish/IMG-20241018-WA0014.jpg
encoded image: 8
encoded image: 9
encoded image: 10
No face found in image: cropped_dataset/Sathwik/IMG-20241028-WA0024.jpg
encoded image: 11
encoded image: 12
encoded image: 13
No face found in image: cropped_dataset/Sathwik/IMG-20241028-WA0022.jpg
encoded image: 14
encoded image: 15
encoded image: 16
encoded image: 17
encoded image: 18
encoded image: 19
encoded image: 20
encoded image: 21
encoded image: 22
encoded image: 23
encoded image: 24
encoded image: 25
encoded image: 26
encoded image: 27
encoded image: 28
encoded image: 29
encoded image: 30
No face found in image: cropped_dataset/Bhavyasri/Polish_20241029_013429050.jpg
encoded image: 31
No face found in image: cropped_dataset/Bhavyasri/IMG_20241028_212832.

In [8]:
import os
import pickle

# Save or load encodings
if not os.path.exists("encodings.pkl"):
    encodeListKnown = findEncodings(images)
    with open("encodings.pkl", "wb") as f:
        pickle.dump((encodeListKnown, classNames), f)
    print("Encodings computed and saved to file.")
else:
    with open("encodings.pkl", "rb") as f:
        encodeListKnown, classNames = pickle.load(f)
    print("Encodings loaded from file.")

encoded image: 97
encoded image: 98
encoded image: 99
encoded image: 100
encoded image: 101
encoded image: 102
encoded image: 103
No face found in image: cropped_dataset/Manish/IMG-20241018-WA0016.jpg
No face found in image: cropped_dataset/Manish/IMG-20241018-WA0014.jpg
encoded image: 104
encoded image: 105
encoded image: 106
No face found in image: cropped_dataset/Sathwik/IMG-20241028-WA0024.jpg
encoded image: 107
encoded image: 108
encoded image: 109
No face found in image: cropped_dataset/Sathwik/IMG-20241028-WA0022.jpg
encoded image: 110
encoded image: 111
encoded image: 112
encoded image: 113
encoded image: 114
encoded image: 115
encoded image: 116
encoded image: 117
encoded image: 118
encoded image: 119
encoded image: 120
encoded image: 121
encoded image: 122
encoded image: 123
encoded image: 124
encoded image: 125
encoded image: 126
No face found in image: cropped_dataset/Bhavyasri/Polish_20241029_013429050.jpg
encoded image: 127
No face found in image: cropped_dataset/Bhavyasr

In [9]:
from datetime import datetime
import os

print("Attendance marking function start")

# Generate a unique filename using the current date and time
now = datetime.now()
date_str = now.strftime('%Y-%m-%d_%H-%M-%S')
attendance_file = f'Attendance_{date_str}.csv'

def markAttendance(name):
    # Check if the attendance file exists, create it with a header if not
    if not os.path.isfile(attendance_file):
        with open(attendance_file, 'w') as f:
            f.write('Name,Time\n')  # Create the header

    with open(attendance_file, 'a+') as f:
        f.seek(0)  # Move to the beginning of the file to read
        myDateList = f.readlines()
        nameList = [line.split(',')[0] for line in myDateList[1:]]  # Skip header to get names only
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%H:%M:%S')
            f.write(f'{name},{dtString}\n')  # Append attendance

print("Function end")


Attendance marking function start
Function end


In [10]:
className = makeClasses()
unique_ClassNames = set(classNames)  # Set of unique class names
# print(classNames)
# print(unique_ClassNames)

def model_summary(encodeListKnown, unique_ClassNames):
    print("Model Summary:")
    print("--------------------")
    print(f"Number of faces encoded: {len(encodeListKnown)}")
    print(f"Encoding vector dimension: {len(encodeListKnown[0]) if encodeListKnown else 'N/A'}")
    print(f"All class names: {list(unique_ClassNames)}")  # Print all unique class names
    print("--------------------")

model_summary(encodeListKnown, unique_ClassNames)

Model Summary:
--------------------
Number of faces encoded: 96
Encoding vector dimension: 128
All class names: ['Manish', 'Gaurav', 'Anand', 'Rupak', 'Sushmita', 'Sathwik', 'Mythili', 'Priyanshu', 'Sandeep', 'Karan', 'Heriv', 'Harsh', 'Bhavyasri']
--------------------


In [ ]:
# Initialize the webcam capture
import cv2
import numpy as np
import face_recognition

print("Starting capture...")
cap = cv2.VideoCapture(0)
Student_Set = set()

# Number of iterations for averaging
num_iterations = 20
distance_records = []
# counter = 0

while True:
    counter += 1
    success, img = cap.read()
    if not success:
        print("Failed to capture image.")
        break
    
    imgSmall = cv2.resize(img, (0, 0), fx=0.25, fy=0.25)
    imgSmall = cv2.cvtColor(imgSmall, cv2.COLOR_BGR2RGB)

    # Detect faces and encode them
    facesCurrFrame = face_recognition.face_locations(imgSmall)
    encodesCurrFrame = face_recognition.face_encodings(imgSmall, facesCurrFrame)

    for encodeFace, faceLoc in zip(encodesCurrFrame, facesCurrFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        matchIndex = np.argmin(faceDis)

        # Store distances for averaging
        distance_records.append(faceDis[matchIndex])

        # Keep only the last `num_iterations` distances
        if len(distance_records) > num_iterations:
            distance_records.pop(0)

        # Calculate the average distance
        average_distance = np.mean(distance_records)
        print(average_distance)
        # Set your recognition threshold
        threshold = 0.4  # You can adjust this value based on your needs

        if matches[matchIndex] and average_distance < threshold:
            name = classNames[matchIndex].upper()
            Student_Set.add(name)

            # Draw rectangle around recognized face
            y1, x2, y2, x1 = faceLoc
            cv2.rectangle(img, (x1*4, y1*4), (x2*4, y2*4), (0, 255, 0), 2)
            cv2.rectangle(img, (x1*4, y2*4 - 35), (x2*4, y2*4), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1*4 + 6, y2*4 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            markAttendance(name)

    # Show the captured image with any recognized face
    cv2.imshow('Webcam', img)
    
    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

print(Student_Set)

# Release the capture and destroy all OpenCV windows
cap.release()
cv2.destroyAllWindows()

Starting capture...
0.19984176739417117
0.20836857581064963
0.2161801192168085
0.22082119895568458
0.22366896933982328
0.2245784485097496
0.2251234506581165
0.22930911014924296
0.23295829499431614
0.234436490375363
0.2358804075294974
0.2384038781395719
0.23895271856239042
0.23866781315498073
0.23939340373741572
0.2393520743661622
0.2390071646755342
0.23916165325543087
0.23953092061713716
0.2397088926457604
0.241751435181008
0.24282255235144068
0.24316222827509332
0.24376676607748937
0.24396648900045298
0.24475464131742936
0.24571640736464267
0.24493982598272973
0.24377033514217633
0.24338411774674476
0.2432690289999273
0.24264273894560429
0.242693497791218
0.24308154201107227
0.24278514823282774
0.24313247496448512
0.24314634568885438
0.24323682540783603
0.2428830305845598
0.24301350115191683
0.24346090405917095
0.2434613800168226
0.2429229270413785
0.24233824920502486
0.24241923356712944
0.24194577687536883
0.24156154671970254
0.2412740052210872
0.24202365268381137
0.2422883355605439
